In [28]:
import numpy as np 
import matplotlib.pyplot as plt 
import pymultinest as pmn
import glob
import sys
import os
import corner


from astropy.cosmology import wCDM


In [29]:
#define all the data here (currently SN only)
data_path = '../jla_data/'
jla_bins = np.loadtxt(data_path+'dist_binned.txt')
cov_mat = np.loadtxt(data_path+'covmat_binned.txt')

vs = np.vstack([sorted(jla_bins[:,0]), sorted(jla_bins[:,1])]).T
np.savetxt('../jla_data/dist_binned.txt', vs)

cov_mat*=1e-6
Cinv = np.linalg.inv(cov_mat)


In [30]:
#define the likelihood and the prior in this cell
#currently only for wCDM


def llhood(model_param, ndim, npar):
    """
    This is the function where we define the log likelihood 
    log(Lhood) = -0.5*chisquare
    """
    
    om, w, h0, M = [model_param[i] for i in xrange(4)]
    wc = wCDM(h0, om, 1-om, w0=w)
    dl_mpc = wc.luminosity_distance(jla_bins[:,0]).value
    mu_th = 5*np.log10(dl_mpc) + 25.
    dif_arr = mu_th - jla_bins[:,1] + M
    chisq = np.dot(dif_arr.T, np.dot(Cinv, dif_arr))
    return -0.5*chisq

def prior(cube, ndim, npar):
    """
    Define the prior for each parameter
    i.e. before doing the experiment what do we know about these parameters
    """
    cube[0]  = cube[0]*1.
    cube[1]  = cube[1]*4. - 2.
    cube[2]  = cube[2]*50. + 50.
    cube[3]  = cube[3]*4. - 2.

In [27]:
#directory where the chains are 
chain_dir = 'chains/'

if not os.path.exists(chain_dir):
    os.makedirs(chain_dir)

#running the nested sampling algorithm
#the verbose argument is to see how the sampler is progressing 
pmn.run(llhood, prior, 4, verbose=True, outputfiles_basename=chain_dir+'jla_wcdm-')

In [ ]:
#a quick plotting function
data = np.loadtxt(chain_dir+'jla_wcdm-')
corner.corner(data, smooth=1, labels=['om', 'w', 'h0', 'M', 'llhood'], color='b')
